In [13]:
%run preprocecssing_helpers.ipynb
import os
import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

ERROR:root:File `'preprocecssing_helpers.ipynb.py'` not found.


In [14]:
path = 'F:\StroopEEG'
eeglab_files = os.listdir(path)
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
eeglab_files

['F:\\StroopEEG\\SA20131216\\20131216_1310.set',
 'F:\\StroopEEG\\SA20131216\\20131216_1441.set',
 'F:\\StroopEEG\\SA20140205\\20140205_1114.set',
 'F:\\StroopEEG\\SA20140205\\20140205_1230.set',
 'F:\\StroopEEG\\SA20140206\\20140206_1029.set',
 'F:\\StroopEEG\\SA20140206\\20140206_1151.set',
 'F:\\StroopEEG\\SA20140207\\20140207_1146.set',
 'F:\\StroopEEG\\SA20140207\\20140207_1219.set',
 'F:\\StroopEEG\\SA20140217\\20140217_1136.set',
 'F:\\StroopEEG\\SA20140217\\20140217_1211.set',
 'F:\\StroopEEG\\SA20140219\\20140219_1046.set',
 'F:\\StroopEEG\\SA20140219\\20140219_1205.set',
 'F:\\StroopEEG\\SA20140220\\20140220_1009.set',
 'F:\\StroopEEG\\SA20140220\\20140220_1123.set',
 'F:\\StroopEEG\\SA20140225\\20140225_1422.set',
 'F:\\StroopEEG\\SA20140225\\20140225_1552.set',
 'F:\\StroopEEG\\SA20140227A\\20140227_1058.set',
 'F:\\StroopEEG\\SA20140227A\\20140227_1220.set',
 'F:\\StroopEEG\\SA20140227B\\20140227_1410.set',
 'F:\\StroopEEG\\SA20140304\\20140304_1418.set',
 'F:\\StroopEEG\\

In [15]:
epoch_destination = 'D:\EpochedEEG'
os.chdir(epoch_destination)

In [16]:
event_id = {'trail_start':  31, 
            'left_button':  131, 
            'right_button': 132,
            'congruent':    133,
            'incongruent':  134,
            'correct':      129,
            'incorrect':    130,
            # targets
            't_baby' :        135,
            't_bell':         136,
            't_bird':         137,
            't_burp':         138,
            't_dog':          139,
            't_drum':         140,
            't_knock':        141,
            't_laugh':        142,
            't_phone':        143,
            't_train':        144,
            't_water':        145,
            #flankers
            'f_baby' :        155,
            'f_bell':         156,
            'f_bird':         157,
            'f_burp':         158,
            'f_dog':          159,
            'f_drum':         160,
            'f_knock':        161,
            'f_laugh':        162,
            'f_phone':        163,
            'f_train':        164,
            'f_water':        165,
}
# event codes 1 - 24 represent flanker and target stim codes. They are NOT explicity defined in dissertation.
for i in range(1,13):
    event_id["flanker_stim_{0}".format(i)] = i
    
for j in range(13, 25):
    event_id["target_stim_{0}".format(j)] = j

In [19]:
for set_file in eeglab_files:
    try:
        start = time.time()
        raw = mne.io.read_raw_eeglab(set_file, preload=True)
        raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
        raw.filter(5,100, fir_design='firwin')
        raw.resample(256, n_jobs=10)
        ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
        ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
        ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
                     ecg_score_func='pearsonr', skew_criterion=None, 
                     kurt_criterion=None, var_criterion=None,add_nodes=None)
        raw_ica = ica.apply(raw)

        m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)
        epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)
        epoch.save(set_file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
        end = time.time()
        print("SUCESSFULLY LOADED time elapsed: {0}", (end - start))
    except :
        print("***Error converting file: {0}***".format(set_file))
        

Reading F:\StroopEEG\SA20131216\1.fdt
Reading 0 ... 929277  =      0.000 ...  1814.994 secs...
***Error converting file: F:\StroopEEG\SA20131216\20131216_1310.set***
Reading F:\StroopEEG\SA20131216\20131216_1441.fdt
Reading 0 ... 1406417  =      0.000 ...  2746.908 secs...
Setting up band-stop filter
Filter length of 3379 samples (6.600 sec) selected
Setting up band-pass filter from 5 - 1e+02 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 845 samples (1.650 sec) selected
5798 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   21.3s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   44.7s finished


5425 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128
Fitting ICA took 125.5s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
5425 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 5424 events and 180 original time points ...
0 bad

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    7.3s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   21.7s finished


5379 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128


...

Setting up band-stop filter
Filter length of 3379 samples (6.600 sec) selected
Setting up band-pass filter from 5 - 1e+02 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 845 samples (1.650 sec) selected
3565 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  31 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 155 156 157 158 159 160 162 163 164 165]


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    9.6s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   17.9s finished


3332 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  31 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 155 156 157 158 159 160 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128
Fitting ICA took 300.8s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
3332 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  31 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 155 156 157 158 159 160 162 163 164 165]
***Error converting file: F:\StroopEEG\SA20140430\20140430_1309.set***
Reading F:\StroopEEG\SA20140430\20140430_1501.fdt
Reading 0 ... 1163169  =      0.000 ...  2271.814 secs...
Setting up band-stop filter
Fil

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   16.2s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   34.9s finished


7207 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128
Fitting ICA took 930.8s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
7207 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
7207 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 7207 events and 180 original time points ...
0 bad epochs 

In [ ]:
start = time.time()
raw = mne.io.read_raw_eeglab(set_file, preload=True)
raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
raw.filter(5,100, fir_design='firwin')
raw.resample(256, n_jobs=10)
ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
             ecg_score_func='pearsonr', skew_criterion=None, 
             kurt_criterion=None, var_criterion=None,add_nodes=None)
raw_ica = ica.apply(raw)

m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)
epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)
end = time.time()
print("time elapsed: {0}", (end - start))

## check if saving and loading epoch file works.

In [ ]:
os.chdir(epoch_destination)

In [ ]:

epoch.save('20131216_1310_epo.fif',verbose = True)

In [ ]:
epoch_files = os.listdir(epoch_destination)

In [ ]:
epoch_file = epoch_files[0]
epoch_ = mne.read_epochs(epoch_file,preload=True)

In [ ]:
epoch_.plot(title="epochs");

In [12]:
for set_file in eeglab_files:
    print(set_file.split('\\')[-1].split('.')[0] + "_epo.fif")

20131216_1310
20131216_1441
20140205_1114
20140205_1230
20140206_1029
20140206_1151
20140207_1146
20140207_1219
20140217_1136
20140217_1211
20140219_1046
20140219_1205
20140220_1009
20140220_1123
20140225_1422
20140225_1552
20140227_1058
20140227_1220
20140227_1410
20140304_1418
20140304_1614
20140306_1133
20140306_1338
20140313_1439
20140313_1630
20140314_1424
20140314_1553
20140318_1043
20140318_1211
20140319_1043
20140319_1342
20140320_1040
20140320_1156
20140320_1421
20140320_1543
20140324_1441
20140324_1553
20140325_1007
20140325_1144
20140327_1022
20140327_1241
20140331_1753
20140402_1603
20140402_1734
20140404_1418
20140404_1628
20140410_1217
20140410_1356
20140411_0929
20140411_1105
20140415_1137
20140415_1309
20140418_1421
20140418_1546
20140423_1404
20140423_1554
20140424_1002
20140424_1606
20140428_0959
20140428_1215
20140430_1309
20140430_1501
